In [2]:
import pandas as pd 
import requests
import xmltodict 
import xlsxwriter


In [216]:
###### BOOKING CREATED ON #######

In [22]:
url = "https://www.eetglobal.com/wsExportacion/wsBookings.asmx?op=getBookingList"



headers= {'content-type': 'text/xml','SOAPAction':'http://juniper.es/getBookingList',"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0",
    "Accept-Encoding": "*",
    "Connection": "keep-alive"}


body = f"""<?xml version="1.0" encoding="utf-8"?>
    <soap:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/">
      <soap:Body>
        <getBookingList xmlns="http://juniper.es/">
          
          
          
          <user>amr.ahmed</user>
          <password>Eetglobal123456@</password>
          
          <BookingDateFrom>20230913</BookingDateFrom>
          <BookingDateTo>20230913</BookingDateTo>
          <Status>ok,con</Status>
        
        
        </getBookingList>
      </soap:Body>
    </soap:Envelope>"""

In [23]:
########BOOKING CHECK IN  ######################

In [24]:
url = "https://www.eetglobal.com/wsExportacion/wsBookings.asmx?op=getBookingList"



headers= {'content-type': 'text/xml','SOAPAction':'http://juniper.es/getBookingList',"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0",
    "Accept-Encoding": "*",
    "Connection": "keep-alive"}


body = f"""<?xml version="1.0" encoding="utf-8"?>
    <soap:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/">
      <soap:Body>
        <getBookingList xmlns="http://juniper.es/">
          
          
          
          <user>amr.ahmed</user>
          <password>Eetglobal123456@</password>
          
          <BeginTravelDateFrom>20230915</BeginTravelDateFrom>
          <BeginTravelDateTo>20230915</BeginTravelDateTo>
          <Status>ok,con</Status>
        
        
        </getBookingList>
      </soap:Body>
    </soap:Envelope>"""


In [25]:
response= requests.post(url=url , data=body , headers=headers)
response_dict= xmltodict.parse(response.text)

In [26]:
booking_list=response_dict['soap:Envelope']['soap:Body']['getBookingListResponse']['getBookingListResult']['wsResult']['BookingList']['Booking']

In [27]:
df=pd.DataFrame(booking_list)
df

,@Id,@Status,@BookingCode,@BookingDate,@LastModifiedDate,@AgencyRef,@TimeZone
0,803099,OK,CXVJ3B,2023-02-08T23:29:35,2023-02-09T16:15:11,108792792,03:00
1,839701,Con,CYJN73,2023-05-13T15:31:52,2023-09-14T11:28:00,,03:00
2,839829,OK,JWK2VL,2023-05-13T23:19:19,2023-05-15T10:16:11,H230513805252,03:00
3,840050,OK,ZG35RF,2023-05-14T18:26:42,2023-05-15T10:40:29,Book_BHTR_2468179,03:00
4,850428,OK,XR6557,2023-06-07T15:48:38,2023-06-07T16:44:24,109868636,03:00
...,...,...,...,...,...,...,...
172,899891,OK,BQKG6R,2023-09-14T12:01:40,2023-09-14T12:01:41,TCH21015,03:00
173,899902,OK,XWV58D,2023-09-14T12:09:08,2023-09-14T12:57:46,,03:00
174,899904,OK,5D5BX7,2023-09-14T12:11:24,2023-09-14T12:11:27,INDUKUMAR-2023-09-14,03:00
175,899909,OK,6PGZMH,2023-09-14T12:19:57,2023-09-14T12:19:59,1_238-23330669_1,03:00


In [28]:
booking_codes_list=df['@BookingCode'].tolist()


In [29]:
def passenger_name(booking):
    holder_res = booking['Holder']
    passenger_name =  holder_res['NameHolder'] + " " + holder_res['LastName']
    return passenger_name      



def get_nights(line):
    nights = line.get('NightsNumber')
    return nights


def get_status_code(lines):
    status = lines['@Status']
    return status



def nationality(booking):
    nationality=booking['Holder']["Nacionalidad"]['#text']
    return nationality


def get_state(lines):
    state=lines['Zone'].get('state')
    return state


def get_country(lines):
    country=lines['Zone'].get('country')
    return country
    

    
def get_sales_price(lines):
    sales_price=lines["sellingprice"]
    return sales_price




def get_profit(lines):
    profit=lines["profit"]
    return profit

def get_customar_remarkes(booking, lines):
    customar_remarkes = booking['Remarks']
    if customar_remarkes== None:
        return lines['Remarks']   
    else:
        return customar_remarkes



#def get_customar_remarkes(booking):
    #customar_remarkes = booking['Remarks']
    #return customar_remarkes



def get_NonRefundable(lines):
    NonRefundable = lines['@NonRefundable']
    return NonRefundable


def get_Description_HOTEL_NAME(lines):
    HOTEL_NAME = lines['ServiceName']
    return HOTEL_NAME


def get_producttypename(lines):
    producttypename = lines['ProductTypeName']
    return producttypename


def get_productgroupname(lines):
    productgroupname = lines['ProductGroupName']
    return productgroupname


def get_Agency(booking):
    Agency = booking['Customer']['Name']
    return Agency


def get_Payment_Type(booking):
    Payment_Type = booking['TPVTransaction']['PaymentType']['#text']
    return Payment_Type


def pax_No(line):
    return line['paxnumber']

def end_date(line):
    return line['EndTravelDate'][:10]


def start_date(line):
    return line['BeginTravelDate'][:10]
    
def LineCancellationChargesDate(line):
    return line['@LineCancellationChargesDate']


def getbookinglinedata(line):
    return line['@LineDate']


def channle(Bookings):
    b=Bookings['Booking']['@Channel']
    return b


def get_board_type(line):
    R=line.get('roomlist')
    if R==None:
        return None , None
    else:
        board_type=[]
        rooms=R['room']
        try : #multirooms
            for room in rooms:
                board_type.append(room['boardtype']['#text'])
        except:
            board_type.append(rooms['boardtype']['#text'])
        v=len(board_type)
        return board_type ,v   
    
    
def get_NO_rooms(lines):
    R=lines.get('roomlist')
    if R==None:
        pass
    else:
        return len(R['room'])
    


In [30]:
def data_retrive(line):
    data_dict={}
    try : #multi lines
            for line in lines:
                print(booking_code)
                data_dict['Booking code']=booking_code
                data_dict['passenger_name']=passenger_name(booking)
                data_dict['No.of Nights']=get_nights(line)
                data_dict['bookinglinedata']=getbookinglinedata(line)
                data_dict['start_date']=start_date(line)
                data_dict['end_date']=end_date(line)
                data_dict['Description_HOTEL_NAME']=get_Description_HOTEL_NAME(line)
                data_dict['Payment_Type']=get_Payment_Type(booking)
                data_dict['producttypename']=get_producttypename(line)
                data_dict['channle']=channle(Bookings)
                data_dict['product group']=get_productgroupname(line)
                data_dict['status']=get_status_code(line)
                data_dict['nationality']=nationality(booking)
                data_dict['zone1']=get_country(line)
                data_dict['zone2']=get_state(line)
                #data_dict['sales price']=get_sales_price(line)
                #data_dict['profit price']=get_profit(line)
                data_dict['customar_remarkes']=get_customar_remarkes(booking, line)
                data_dict['NonRefundable']=get_NonRefundable(line)
                data_dict['Agency']=get_Agency(booking)
                #data_dict['NO_rooms']=get_NO_rooms(line)
                data_dict['board_type'],data_dict['NO_rooms']=get_board_type(line)[0], get_board_type(line)[1] 
                f_list.append(data_dict)
    except: #one line
        print(booking_code)
        x=lines
        data_dict['Booking code']=booking_code
        data_dict['passenger_name']=passenger_name(booking)
        data_dict['No.of Nights']=get_nights(x)
        data_dict['bookinglinedata']=getbookinglinedata(x)
        data_dict['start_date']=start_date(x)
        data_dict['end_date']=end_date(x)
        data_dict['Description_HOTEL_NAME']=get_Description_HOTEL_NAME(x)
        data_dict['Payment_Type']=get_Payment_Type(booking)
        data_dict['producttypename']=get_producttypename(x)
        data_dict['channle']=channle(Bookings)
        data_dict['product group']=get_productgroupname(x)
        data_dict['status']=get_status_code(x)
        data_dict['nationality']=nationality(booking)
        data_dict['zone1']=get_country(x)
        data_dict['zone2']=get_state(x)
        #data_dict['sales price']=get_sales_price(lines)
        #data_dict['profit price']=get_profit(lines)
        data_dict['customar_remarkes']=get_customar_remarkes(booking, lines)
        data_dict['NonRefundable']=get_NonRefundable(x)
        data_dict['Agency']=get_Agency(booking)
        #data_dict['NO_rooms']=get_NO_rooms(x)
        data_dict['board_type'],data_dict['NO_rooms']=get_board_type(x)[0], get_board_type(x)[1]
        f_list.append(data_dict)
    return f_list

In [31]:
f_list=[]

In [32]:
for booking_code in booking_codes_list:
    url = "https://www.eetglobal.com/wsExportacion/wsBookings.asmx?op=getBookings"


    headers= {'content-type': 'text/xml','SOAPAction':"http://juniper.es/getBookings","User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0",
                "Accept-Encoding": "*","Connection": "keep-alive"}


    body = f"""<?xml version="1.0" encoding="utf-8"?>
            <soap:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:soap="http://schemas.xmlsoap.org/soap/envelope/">
              <soap:Body>
                <getBookings xmlns="http://juniper.es/">
                  <user>amr.ahmed</user>
                  <password>Eetglobal123456@</password>
                  <BookingCode>{booking_code}</BookingCode>
                </getBookings>
              </soap:Body>
            </soap:Envelope>"""
    response=requests.post(url=url , headers=headers , data=body)
    booking_info_dict=xmltodict.parse(response.text)
    Bookings=booking_info_dict['soap:Envelope']['soap:Body']['getBookingsResponse']['getBookingsResult']['wsResult'].get('Bookings')

    
    if Bookings==None:
        #print("not excist")
        pass 

    else:
        booking= booking_info_dict['soap:Envelope']['soap:Body']['getBookingsResponse']['getBookingsResult']['wsResult']['Bookings']['Booking']
        lines=booking['Lines']['Line']
        data_retrive(lines)
    #lines=booking_info_dict['soap:Envelope']['soap:Body']['getBookingsResponse']['getBookingsResult']['wsResult']['Bookings']['Booking']['Lines']['Line']
    #print(booking_code)

CXVJ3B
CXVJ3B
CYJN73
CYJN73
JWK2VL
JWK2VL
ZG35RF
ZG35RF
XR6557
XR6557
YNHKG3
YNHKG3
CRQ4R9
CRQ4R9
5ZB5BT
5ZB5BT
TJ1YCT
TJ1YCT
75V53M
75V53M
1SNJGS
1SNJGS
FM4J4B
FM4J4B
M5J8S2
M5J8S2
8W9P1V
8W9P1V
L473L2
L473L2
4J4NM1
4J4NM1
TVZR1W
TVZR1W
95GYG3
95GYG3
Z5BYM1
Z5BYM1
2GTCT6
2GTCT6
2L51GL
2L51GL
64X654
64X654
R64ZXF
R64ZXF
QMF42Z
QMF42Z
9R1S3H
9R1S3H
W183H6
W183H6
X4WCV7
X4WCV7
9X4QGS
9X4QGS
B2K3L8
B2K3L8
CNQ1NP
CNQ1NP
GY4VCR
GY4VCR
PTN5R6
PTN5R6
4TXK5N
4TXK5N
55K4FK
55K4FK
1ZD1QC
1ZD1QC
V13CSV
V13CSV
9Q4J5C
9Q4J5C
K1S9PY
K1S9PY
W6GL6K
W6GL6K
QFGMH1
QFGMH1
M9BL52
M9BL52
551Z5Q
551Z5Q
RGNN9W
RGNN9W
58V85Y
58V85Y
F9H18M
F9H18M
5WX734
5WX734
231HFJ
231HFJ
DMV3GF
DMV3GF
3LB34G
3LB34G
HSKTC1
HSKTC1
KL1QF1
KL1QF1
QJ9C9L
QJ9C9L
7PQMX4
7PQMX4
217FMS
217FMS
MTY8K2
MTY8K2
4M75KT
4M75KT
GM1KQ4
GM1KQ4
X2RMVW
X2RMVW
WWCX79
WWCX79
321YHK
321YHK
R4481P
R4481P
CTQN4F
CTQN4F
QRJP4F
QRJP4F
8SB2MP
8SB2MP
HL1MNW
HL1MNW
441523
441523
2JB2WP
2JB2WP
7CCMKN
7CCMKN
8W4ZH4
8W4ZH4
Y12SR9
Y12SR9
4L5C1K
4L5C1K
D5C8D4

In [33]:
DF=pd.DataFrame(f_list)

In [34]:
DF

,Booking code,passenger_name,No.of Nights,bookinglinedata,start_date,end_date,Description_HOTEL_NAME,Payment_Type,producttypename,channle,product group,status,nationality,zone1,zone2,customar_remarkes,NonRefundable,Agency,board_type,NO_rooms
0,CXVJ3B,Ralf Klein,11,2023-02-08T23:29:35,2023-09-15,2023-09-26,LABRANDA Club Makadi~,Credit,Accommodation,XmlU,Dynamic Egypt (Labranda),OK,GERMANY,Egypt,Hurghada,hotel confirmation number ( 84975),True,AurumTours GmbH,[Hard All inclusive],1.0
1,CYJN73,Reham Saleh,1,2023-05-13T15:31:52,2023-09-15,2023-09-16,"Robert Treat Hotel, BW Premier Collection",Pre-payment,DerbySoftV4,Web2,HCC - DerbySoft Best Western,OK,SAUDI ARABIA,USA,New York,hotel confirmation number 358303091,False,Global International,[Room only],1.0
2,JWK2VL,Muhannad Rashwan,3,2023-05-13T23:19:20,2023-09-15,2023-09-18,Pavilion Hotel Kuala Lumpur,Credit,Accommodation,XmlU,Hotels Malaysia,OK,SAUDI ARABIA,Malaysia,Bukit Bintang,hotel confirmation number:237377,False,Seera Non-KSA (Enigma),[Bed and Breakfast],1.0
3,ZG35RF,MUDHHI ALSULAMI,3,2023-05-14T18:26:42,2023-09-15,2023-09-18,Grand Millennium Kuala Lumpur,Credit,Accommodation,XmlU,Hotels Malaysia,OK,SAUDI ARABIA,Malaysia,Bukit Bintang,hotel confirmation number:26012545,False,WITHIN EARTH HOLIDAYS,[Bed and Breakfast],1.0
4,XR6557,Annett Gabriele Rönsch,10,2023-06-07T15:48:38,2023-09-15,2023-09-25,Steigenberger aqua Magic hotel~,Credit,Accommodation,XmlU,Dynamic Egypt (Steigenberger hotels),OK,GERMANY,Egypt,Hurghada,Hotel confirmation number # 2240695,True,AurumTours GmbH,[Soft All Inclusive],1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173,BQKG6R,Dalal ALMUTAIRI,4,2023-09-14T12:01:40,2023-09-15,2023-09-19,Lancaster Eden Bay Hotel,Credit,Accommodation,XmlU,EET DMC (Exclusive Services),OK,KUWAIT,Lebanon,Beirut,None,False,Jaunt In Time (Tripnclick),[Room Only],1.0
174,XWV58D,ABDULLAH ALSADDI,2,2023-09-14T12:09:08,2023-09-15,2023-09-17,Crystal Inn by Almasa~,Credit,Accommodation,XmlU,Dynamic Egypt,OK,SAUDI ARABIA,Egypt,Alamein,None,True,Travel Designer DMCC (Rezlive),[Bed and Breakfast],1.0
175,5D5BX7,INDUKUMAR VASUDEVAN,1,2023-09-14T12:11:24,2023-09-15,2023-09-16,Millennium Downtown Abu Dhabi (ex.Crowne Plaza...,Credit,Accommodation,XmlU,EET GLOBAL,OK,UNITED ARAB EMIRATES,United Arab Emirates,Abu Dhabi city,None,False,Flyin.com (Saudi Ebreez),[Bed and Breakfast],1.0
176,6PGZMH,Mr. AHMEDHAFEZAMIN ALY,10,2023-09-14T12:19:58,2023-09-15,2023-09-25,Crown Rose Hotel~,Credit,Accommodation,XmlU,KSA Dynamic VAT,OK,EGYPT,Saudi Arabia,Riyadh City,None,True,OTTILA INTERNATIONAL PVT LTD,[Bed and Breakfast],1.0


In [35]:
EGY=DF[DF['zone1']=='Egypt']

In [36]:
EGY

,Booking code,passenger_name,No.of Nights,bookinglinedata,start_date,end_date,Description_HOTEL_NAME,Payment_Type,producttypename,channle,product group,status,nationality,zone1,zone2,customar_remarkes,NonRefundable,Agency,board_type,NO_rooms
0,CXVJ3B,Ralf Klein,11,2023-02-08T23:29:35,2023-09-15,2023-09-26,LABRANDA Club Makadi~,Credit,Accommodation,XmlU,Dynamic Egypt (Labranda),OK,GERMANY,Egypt,Hurghada,hotel confirmation number ( 84975),True,AurumTours GmbH,[Hard All inclusive],1.0
4,XR6557,Annett Gabriele Rönsch,10,2023-06-07T15:48:38,2023-09-15,2023-09-25,Steigenberger aqua Magic hotel~,Credit,Accommodation,XmlU,Dynamic Egypt (Steigenberger hotels),OK,GERMANY,Egypt,Hurghada,Hotel confirmation number # 2240695,True,AurumTours GmbH,[Soft All Inclusive],1.0
10,1SNJGS,Gaby Jähner,14,2023-07-18T13:00:17,2023-09-15,2023-09-29,Sunrise Holidays Resort (Adults Only),Credit,Accommodation,XmlU,Hotels Egypt VAT,OK,GERMANY,Egypt,Hurghada,hotel confirmation number 1080638,False,AurumTours GmbH,[Hard All inclusive],1.0
19,2GTCT6,Anna Merle Schiemann,10,2023-08-13T22:24:25,2023-09-15,2023-09-25,LABRANDA Royal Makadi~,Credit,Accommodation,XmlU,Dynamic Egypt (Labranda),OK,GERMANY,Egypt,Hurghada,"Confirmation number : 128,748",True,AurumTours GmbH,[Hard All inclusive],1.0
46,5WX734,ABDULAZIZ AKRAM,5,2023-08-28T17:35:30,2023-09-15,2023-09-20,Pyramisa Suites Hotel Cairo,Credit,Accommodation,XmlU,Hotels Egypt VAT,OK,SAUDI ARABIA,Egypt,Cairo,Hotel Confirmation Number #280342,False,Rakuten Travel Xchange Pte Ltd,[Bed and Breakfast],1.0
51,KL1QF1,Pakistani Group,4,2023-08-29T11:26:19,2023-09-15,2023-09-19,Maritim Jolie Ville Resort Casino Sharm El Sh...,Pre-payment,Budget,Web2,Hotels BTB,OK,EGYPT,Egypt,Sharm El Sheikh,None,False,Individual company ( All Nationality ),None,NaN
54,217FMS,Arahman Alsubaiee,6,2023-08-29T18:56:01,2023-09-15,2023-09-21,Fairmont Nile City Hotel Cairo-,Credit,Accommodation,XmlU,Hotels Egypt VAT,OK,BAHRAIN,Egypt,Cairo,Confirmation Number:\n6952645,True,Wego,[Room Only],1.0
55,MTY8K2,Frank Hofmann,11,2023-08-29T21:21:03,2023-09-15,2023-09-26,Jaz Aquamarine~,Credit,Accommodation,XmlU,EET Levant Dynamic (Jaz Group),OK,GERMANY,Egypt,Hurghada,Hotel Confirmation no.1055295,False,AurumTours GmbH,[Hard All inclusive],1.0
84,4FHX54,Abdulsalam Aldhubayb,1,2023-09-05T03:49:26,2023-09-15,2023-09-16,Hotelux La Playa Alamein~,Credit,Accommodation,XmlU,Dynamic Egypt,OK,SAUDI ARABIA,Egypt,Alamein,Hotel Confirmation no.161663,True,Seera Non-KSA (Enigma),[Bed and Breakfast],1.0
85,4KXKHS,RAFFAELE BENINCASA,7,2023-09-05T13:03:53,2023-09-15,2023-09-22,"Domina Coral Bay Resort, Diving , Spa Casino~",Credit,Accommodation,XmlU,Dynamic Egypt VAT,OK,ITALY,Egypt,Sharm El Sheikh,None,True,Emerging Travel Inc - Ostrovok,[Hard All inclusive],1.0


In [37]:
KSA=DF[(DF['Description_HOTEL_NAME'].str.contains("Raffles Makkah Palace",case=False ))|(DF['Description_HOTEL_NAME'].str.contains("Makkah Clock Royal Tower, A Fairmont Hotel",case=False ))|(DF['Description_HOTEL_NAME'].str.contains("Swissotel Al Maqam Makkah",case=False ))|(DF['Description_HOTEL_NAME'].str.contains("Swissotel Makkah",case=False ))]


In [38]:
KSA.reset_index(drop=True)

,Booking code,passenger_name,No.of Nights,bookinglinedata,start_date,end_date,Description_HOTEL_NAME,Payment_Type,producttypename,channle,product group,status,nationality,zone1,zone2,customar_remarkes,NonRefundable,Agency,board_type,NO_rooms
0,G937WC,Emad Ahmed,2,2023-09-10T16:47:32,2023-09-15,2023-09-17,"Makkah Clock Royal Tower, A Fairmont Hotel",Credit,Accommodation,XmlU,KSA Hotels VAT,OK,EGYPT,Saudi Arabia,Al Hajlah,None,True,Jaunt In Time (Tripnclick),[Bed and Breakfast],1.0


In [39]:
writer = pd.ExcelWriter('Bookings Created on  Egypt &Agreed  KSA Hotels in 13 Sep ,2023.xlsx', engine='xlsxwriter')
EGY.to_excel(writer, sheet_name='EGY', index=False)
KSA.to_excel(writer, sheet_name='KSA', index=False)
writer.close()